<a href="https://colab.research.google.com/github/liminfinity/aiweek/blob/master/aiweek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача 2: Классификатор изображений с дронов

**Задание выполнил:** Полешко Артём

**Команда:** Экспонента

**Что нужно сделать:**
1. Найдите 15-20 аэрофотоснимков (можно с бесплатных источников: Unsplash, Pexels, поиск "aerial view")
2. Используйте vision-модели (GPT-4 Vision, Gemini Pro Vision, Claude 3) для
классификации:
    - Тип местности (город/лес/поле/водоем)
    - Наличие объектов (здания/дороги/транспорт)
    - Краткое описание (1-2 предложения)
3. Сравните результаты, напишите выводы

## Тестовые данные

Для выполнения задачи были выбраны 15 аэрофотоснимков с высоты птичьего полета (aerial views) из Unsplash. Эти изображения представляют разнообразные типы местности: города, леса, поля, водоемы, а также комбинации с объектами, такими как здания, дороги и транспорт. Чтобы обеспечить доступность, они были загружены в репозиторий GitHub.

Базовый URL для доступа: https://github.com/liminfinity/aiweek/blob/master/aerial_views

## Выбранные модели

Для сравнительного анализа были выбраны три vision-модели с бесплатным доступом через платформу OpenRouter.

1. **x-ai/grok-4.1-fast:** быстрая версия модели Grok от xAI.
Документация: https://openrouter.ai/x-ai/grok-4.1-fast

2. **nvidia/nemotron-nano-12b-v2-vl:** компактная vision-модель от NVIDIA.
Документация: https://openrouter.ai/nvidia/nemotron-nano-12b-v2-vl:free

3. **microsoft/phi-4-multimodal-instruct:** мультимодальная модель от Microsoft.
Документация: https://openrouter.ai/microsoft/phi-4-multimodal-instruct

## Код реализации

### Блок 1. Настройка окружения

In [ ]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')  # Ключ для OpenRouter

In [ ]:
!pip install -q -U openai  # Для работы с OpenRouter API
!pip install -q -U requests pandas

In [ ]:
import json
import requests
import re
import time

from openai import OpenAI

import pandas as pd

### Блок 2. Определение класса OpenRouter модели

Иногда модели возвращают JSON с лишними обёртками, поэтому напишем функцию для приведения текста к чистому формату.

In [ ]:
def remove_json_wrappers(text):
    if not text:
        return text

    text = text.strip()
    text = re.sub(r'^```json\s*', '', text)
    text = re.sub(r'\s*```$', '', text)
    text = re.sub(r'^`|`$', '', text)

    return text.strip()

In [ ]:
class OpenRouterModel:
    def __init__(self, name, model_id):
        self.name = name
        self.model_id = model_id
        self.client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=OPENROUTER_API_KEY)

    def analyze_image(self, image_url, prompt):
        start_time = time.time()
        response = self.client.chat.completions.create(
            model=self.model_id,
            messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},  # Текстовый промпт
                    {"type": "image_url", "image_url": {"url": image_url}}  # URL изображения для анализа
                ]
            }]
        )
        end_time = time.time()
        json_resp = json.loads(remove_json_wrappers(response.choices[0].message.content))
        return {"response": json_resp, "time": end_time - start_time}

### Блок 3. Инициализация моделей и подготовка к анализу

In [ ]:
grok = OpenRouterModel("Grok-4.1-Fast", "x-ai/grok-4.1-fast:free")
nemotron = OpenRouterModel("Nemotron-Nano-12B-VL", "nvidia/nemotron-nano-12b-v2-vl:free")
phi = OpenRouterModel("Phi-4-Multimodal", "microsoft/phi-4-multimodal-instruct")

models = [grok, nemotron, phi]

In [ ]:
image_files = [
    "image1.jpg", "image2.jpg", "image3.jpg", "image4.jpg",
    "image5.jpg", "image6.jpg", "image7.jpg", "image8.jpg",
    "image9.jpg", "image10.jpg", "image11.jpg", "image12.jpg",
    "image13.jpg", "image14.jpg", "image15.jpg"
]

base_url = "https://raw.githubusercontent.com/liminfinity/aiweek/master/aerial_views/"

prompt = """
Проанализируй аэрофотоснимок и верни результат в формате JSON.

Структура JSON объекта:
{
    "terrain_type": "тип местности",
    "detected_objects": ["список объектов"],
    "description": "краткое описание"
}

ВАЖНО:
1. Верни ТОЛЬКО сырой JSON без каких-либо обратных кавычек, markdown разметки или дополнительного текста.
2. Все поля должны быть заполнены на русском языке.
"""

results = {}  # Словарь для хранения результатов классификации изображений

Модели могут не ответить с первого раза, поэтому будем повторять запросы до тех пор, пока не получим результат или не достигнем лимита попыток.

In [ ]:
MAX_RETRIES = 3

def safe_analyze(model, image_url, prompt):
    for attempt in range(MAX_RETRIES):
        try:
            return model.analyze_image(image_url, prompt)
        except Exception as e:
            print(f"{model.name} попытка {attempt+1}/{MAX_RETRIES} провалилась:", e)
    return {"response": None, "time": 0}

### Блок 4. Запуск анализа изображений

In [ ]:
for idx, img_file in enumerate(image_files, 1):
    image_url = base_url + img_file
    print(f"Обработка изображения {idx}: {image_url}")

    models_response = {
        model.name: safe_analyze(model, image_url, prompt)
        for model in models
    }

    results[idx] = models_response

Обработка изображения 1: https://raw.githubusercontent.com/liminfinity/aiweek/master/aerial_views/image1.jpg
Nemotron-Nano-12B-VL попытка 1/3 провалилась: Extra data: line 1 column 184 (char 183)
Обработка изображения 2: https://raw.githubusercontent.com/liminfinity/aiweek/master/aerial_views/image2.jpg
Обработка изображения 3: https://raw.githubusercontent.com/liminfinity/aiweek/master/aerial_views/image3.jpg
Обработка изображения 4: https://raw.githubusercontent.com/liminfinity/aiweek/master/aerial_views/image4.jpg
Phi-4-Multimodal попытка 1/3 провалилась: Error code: 429 - {'error': {'message': 'Provider returned error', 'code': 429, 'metadata': {'raw': 'microsoft/phi-4-multimodal-instruct is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations', 'provider_name': 'DeepInfra'}}, 'user_id': 'user_35t9qndxd4RJkuqXLHWORmN6u6Z'}
Phi-4-Multimodal попытка 2/3 провалилась: Error code: 429 - {

### Блок 5. Отображение результатов

In [ ]:
from IPython.display import HTML, display


def show_results(img_id, img_url, models_response):

    html = f"""
    <div style="margin: 30px 0; padding: 20px; border: 2px solid #e0e0e0; border-radius: 15px; background: #f8f9fa;">
        <h2 style="color: #2c3e50; text-align: center; margin-bottom: 20px; font-family: Arial, sans-serif;">Анализ изображения {img_id}</h2>

        <div style="text-align: center; margin-bottom: 25px;">
            <img src="{img_url}" style="max-width: 400px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
        </div>

        <div style="display: flex; flex-direction: column; gap: 15px;">
    """

    for model_name, data in models_response.items():
        if data['response'] and isinstance(data['response'], dict):
            response_data = data['response']

            display_content = f"""
            <div style="font-family: Arial, sans-serif; color: #2d3748; line-height: 1.6;">
                <div style="margin-bottom: 12px;">
                    <span style="font-weight: bold; color: #2c5aa0;">Тип местности:</span>
                    <span style="margin-left: 8px; background: #e8f4fd; padding: 4px 8px; border-radius: 4px;">{response_data.get('terrain_type', 'Не определен')}</span>
                </div>

                <div style="margin-bottom: 12px;">
                    <span style="font-weight: bold; color: #2c5aa0;">Обнаруженные объекты:</span>
                    <div style="margin-left: 8px; margin-top: 4px;">
            """

            objects = response_data.get('detected_objects', [])
            if objects:
                for obj in objects:
                    display_content += f'<span style="background: #e8f4fd; padding: 4px 8px; border-radius: 4px; margin-right: 6px; margin-bottom: 6px; display: inline-block;">{obj}</span>'
            else:
                display_content += '<span style="color: #718096;">Объекты не обнаружены</span>'

            display_content += f"""
                    </div>
                </div>

                <div style="margin-bottom: 12px;">
                    <span style="font-weight: bold; color: #2c5aa0;">Описание:</span>
                    <div style="margin-left: 8px; margin-top: 4px; padding: 8px 12px; background: #f7fafc; border-radius: 6px; border-left: 3px solid #4299e1;">
                        {response_data.get('description', 'Описание отсутствует')}
                    </div>
                </div>
            </div>
            """

        else:
            display_content = """
            <div style="color: #e53e3e; padding: 12px; background: #fed7d7; border-radius: 6px; border-left: 3px solid #e53e3e;">
                ❌ Не удалось получить корректный ответ от модели
            </div>
            """

        time_str = f"{data['time']:.2f}с" if data.get('time') else "N/A"

        html += f"""
            <div style="padding: 20px; border: 1px solid #cbd5e0; border-radius: 10px; background: white; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
                <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px; padding-bottom: 10px; border-bottom: 1px solid #e2e8f0;">
                    <h3 style="color: #2d3748; margin: 0; font-family: Arial, sans-serif; font-size: 18px;">🤖 {model_name.split('/')[-1]}</h3>
                    <div style="display: flex; gap: 10px; align-items: center;">
                        <span style="background: #edf2f7; color: #4a5568; padding: 4px 8px; border-radius: 5px; font-size: 12px; font-weight: bold;">
                            ⏱️ {time_str}
                        </span>
                    </div>
                </div>
                <div style="background: white; color: #2d3748;">
                    {display_content}
                </div>
            </div>
        """

    html += """
        </div>
    </div>
    """

    display(HTML(html))

In [ ]:
for idx, img_url in enumerate(image_files, 1):
    show_results(idx, base_url + img_url, results[idx])

## Оценка качества классификации изображений

Для оценки результатов работы vision-моделей используется следующая балльная система по трём критериям:

### 1. Точность определения типа местности

- **5** — Тип местности определён полностью корректно
- **4** — Незначительные неточности (например, "городская" вместо "урбанизированная")
- **3** — Частично верно, но упущены важные детали
- **2** — Значительные ошибки в определении
- **1** — Полностью неверно или нерелевантно

### 2. Полнота обнаруженных объектов

- **5** — Все ключевые объекты обнаружены
- **4** — Пропущен 1 значимый объект
- **3** — Пропущено 2-3 значимых объекта
- **2** — Обнаружена лишь малая часть объектов
- **1** — Объекты практически не обнаружены

### 3. Детальность и корректность описания

- **5** — Описание подробное, точное и информативное
- **4** — Хорошее описание с незначительными упущениями
- **3** — Базовое описание, упущены важные детали
- **2** — Описание слишком общее или содержит ошибки
- **1** — Описание некорректно или бессмысленно

In [ ]:
# (Изображение, Модель, Точность типа, Полнота объектов, Детальность описания, Время)
data = [
    # Изображение 1 - Лес у озера
    ("Изображение 1", "Grok-4.1-Fast", 5, 4, 5, 13.21),
    ("Изображение 1", "Nemotron-Nano-12B-VL", 3, 3, 3, 6.51),
    ("Изображение 1", "Phi-4-Multimodal", 5, 4, 5, 9.83),

    # Изображение 2 - Городская развязка
    ("Изображение 2", "Grok-4.1-Fast", 5, 5, 5, 16.93),
    ("Изображение 2", "Nemotron-Nano-12B-VL", 5, 5, 4, 10.01),
    ("Изображение 2", "Phi-4-Multimodal", 5, 5, 5, 25.02),

    # Изображение 3 - Городской перекрёсток
    ("Изображение 3", "Grok-4.1-Fast", 5, 5, 5, 14.31),
    ("Изображение 3", "Nemotron-Nano-12B-VL", 5, 5, 4, 6.65),
    ("Изображение 3", "Phi-4-Multimodal", 5, 5, 4, 8.52),

    # Изображение 4 - Лесная автодорожная развязка
    ("Изображение 4", "Grok-4.1-Fast", 5, 5, 5, 13.36),
    ("Изображение 4", "Nemotron-Nano-12B-VL", 3, 2, 2, 5.72),
    ("Изображение 4", "Phi-4-Multimodal", 4, 5, 4, 6.86),

    # Изображение 5 - Песчаный берег океана
    ("Изображение 5", "Grok-4.1-Fast", 5, 4, 5, 19.68),
    ("Изображение 5", "Nemotron-Nano-12B-VL", 5, 3, 4, 10.41),
    ("Изображение 5", "Phi-4-Multimodal", 5, 4, 5, 5.17),

    # Изображение 6 - Прибрежный жилой район с каналами
    ("Изображение 6", "Grok-4.1-Fast", 5, 5, 5, 13.85),
    ("Изображение 6", "Nemotron-Nano-12B-VL", 5, 5, 5, 6.61),
    ("Изображение 6", "Phi-4-Multimodal", 5, 5, 5, 3.46),

    # Изображение 7 - Рисовые поля
    ("Изображение 7", "Grok-4.1-Fast", 5, 5, 5, 16.36),
    ("Изображение 7", "Nemotron-Nano-12B-VL", 3, 2, 2, 6.36),
    ("Изображение 7", "Phi-4-Multimodal", 5, 4, 4, 6.70),

    # Изображение 8 - Пригородная зона
    ("Изображение 8", "Grok-4.1-Fast", 5, 5, 5, 13.65),
    ("Изображение 8", "Nemotron-Nano-12B-VL", 4, 4, 4, 6.99),
    ("Изображение 8", "Phi-4-Multimodal", 5, 5, 4, 2.95),

    # Изображение 9 - Прибрежная зона с пляжем
    ("Изображение 9", "Grok-4.1-Fast", 5, 5, 5, 20.95),
    ("Изображение 9", "Nemotron-Nano-12B-VL", 4, 4, 3, 7.51),
    ("Изображение 9", "Phi-4-Multimodal", 5, 4, 4, 2.63),

    # Изображение 10 - Парламент в Канберре
    ("Изображение 10", "Grok-4.1-Fast", 5, 5, 5, 13.62),
    ("Изображение 10", "Nemotron-Nano-12B-VL", 3, 3, 3, 4.98),
    ("Изображение 10", "Phi-4-Multimodal", 5, 5, 5, 2.89),

    # Изображение 11 - Мост над рекой
    ("Изображение 11", "Grok-4.1-Fast", 5, 5, 5, 17.63),
    ("Изображение 11", "Nemotron-Nano-12B-VL", 4, 4, 4, 10.51),
    ("Изображение 11", "Phi-4-Multimodal", 5, 4, 4, 2.01),

    # Изображение 12 - Городская застройка
    ("Изображение 12", "Grok-4.1-Fast", 5, 5, 5, 11.66),
    ("Изображение 12", "Nemotron-Nano-12B-VL", 5, 5, 4, 10.49),
    ("Изображение 12", "Phi-4-Multimodal", 5, 5, 4, 2.72),

    # Изображение 13 - Тропический пляж
    ("Изображение 13", "Grok-4.1-Fast", 5, 5, 5, 11.27),
    ("Изображение 13", "Nemotron-Nano-12B-VL", 4, 3, 2, 5.97),
    ("Изображение 13", "Phi-4-Multimodal", 5, 5, 4, 2.05),

    # Изображение 14 - Хвойный лес
    ("Изображение 14", "Grok-4.1-Fast", 5, 4, 5, 12.77),
    ("Изображение 14", "Nemotron-Nano-12B-VL", 1, 2, 1, 6.82),
    ("Изображение 14", "Phi-4-Multimodal", 5, 4, 4, 7.02),

    # Изображение 15 - Сельскохозяйственная местность с рекой
    ("Изображение 15", "Grok-4.1-Fast", 5, 5, 5, 17.37),
    ("Изображение 15", "Nemotron-Nano-12B-VL", 3, 2, 2, 8.24),
    ("Изображение 15", "Phi-4-Multimodal", 5, 5, 4, 3.07),
]

In [ ]:
df = pd.DataFrame(data, columns=["Изображение", "Модель", "Точность типа", "Полнота объектов", "Детальность описания", "Время"])
df["Оценка"] = df[["Точность типа", "Полнота объектов", "Детальность описания"]].mean(axis=1).round(2)

In [ ]:
for name, group in df.groupby("Изображение", sort=False):
    print(f"\n{'─' * 80}")
    print(f"{name}")
    print(f"{'─' * 80}")
    display(group[["Модель", "Точность типа", "Полнота объектов", "Детальность описания", "Время", "Оценка"]])


────────────────────────────────────────────────────────────────────────────────
Изображение 1
────────────────────────────────────────────────────────────────────────────────


,Модель,Точность типа,Полнота объектов,Детальность описания,Время,Оценка
0,Grok-4.1-Fast,5,4,5,13.21,4.67
1,Nemotron-Nano-12B-VL,3,3,3,6.51,3.00
2,Phi-4-Multimodal,5,4,5,9.83,4.67



────────────────────────────────────────────────────────────────────────────────
Изображение 2
────────────────────────────────────────────────────────────────────────────────


,Модель,Точность типа,Полнота объектов,Детальность описания,Время,Оценка
3,Grok-4.1-Fast,5,5,5,16.93,5.00
4,Nemotron-Nano-12B-VL,5,5,4,10.01,4.67
5,Phi-4-Multimodal,5,5,5,25.02,5.00



────────────────────────────────────────────────────────────────────────────────
Изображение 3
────────────────────────────────────────────────────────────────────────────────


...

,Модель,Точность типа,Полнота объектов,Детальность описания,Время,Оценка
36,Grok-4.1-Fast,5,5,5,11.27,5.00
37,Nemotron-Nano-12B-VL,4,3,2,5.97,3.00
38,Phi-4-Multimodal,5,5,4,2.05,4.67



────────────────────────────────────────────────────────────────────────────────
Изображение 14
────────────────────────────────────────────────────────────────────────────────


,Модель,Точность типа,Полнота объектов,Детальность описания,Время,Оценка
39,Grok-4.1-Fast,5,4,5,12.77,4.67
40,Nemotron-Nano-12B-VL,1,2,1,6.82,1.33
41,Phi-4-Multimodal,5,4,4,7.02,4.33



────────────────────────────────────────────────────────────────────────────────
Изображение 15
────────────────────────────────────────────────────────────────────────────────


,Модель,Точность типа,Полнота объектов,Детальность описания,Время,Оценка
42,Grok-4.1-Fast,5,5,5,17.37,5.00
43,Nemotron-Nano-12B-VL,3,2,2,8.24,2.33
44,Phi-4-Multimodal,5,5,4,3.07,4.67


In [ ]:
summary = df.groupby('Модель').agg({
    "Точность типа": 'mean',
    "Полнота объектов": 'mean',
    "Детальность описания": 'mean',
    "Оценка": 'mean',
    "Время": ['mean', 'median', 'min', 'max']
}).round(2)

In [ ]:
summary.columns = ['Точность типа', 'Полнота объектов', 'Детальность', 'Оценка',
                   'Время (среднее)', 'Время (медиана)', 'Время (мин)', 'Время (макс)']
display(summary)

,Точность типа,Полнота объектов,Детальность,Оценка,Время (среднее),Время (медиана),Время (мин),Время (макс)
Модель,,,,,,,,
Grok-4.1-Fast,5.00,4.80,5.00,4.93,15.11,13.85,11.27,20.95
Nemotron-Nano-12B-VL,3.80,3.47,3.13,3.47,7.59,6.82,4.98,10.51
Phi-4-Multimodal,4.93,4.60,4.33,4.62,6.06,3.46,2.01,25.02


In [ ]:
best_score = summary['Оценка'].idxmax()
best_time_median = summary['Время (медиана)'].idxmin()

In [ ]:
print(f"\n🏆 Лучшая модель по качеству: {best_score}")
print(f"   Средний балл: {summary.loc[best_score, 'Оценка']:.2f} из 5.00")
print(f"   Медианное время: {summary.loc[best_score, 'Время (медиана)']:.2f} сек")

print(f"\n⚡ Самая быстрая модель: {best_time_median}")
print(f"   Медианное время: {summary.loc[best_time_median, 'Время (медиана)']:.2f} сек")
print(f"   Средний балл: {summary.loc[best_time_median, 'Оценка']:.2f} из 5.00")


🏆 Лучшая модель по качеству: Grok-4.1-Fast
   Средний балл: 4.93 из 5.00
   Медианное время: 13.85 сек

⚡ Самая быстрая модель: Phi-4-Multimodal
   Медианное время: 3.46 сек
   Средний балл: 4.62 из 5.00


## Выводы

### Лучшая модель

***Grok-4.1-Fast*** (средняя оценка 4.93 из 5.00) — лидер по качеству распознавания.

### Точность определения типа местности

***Grok-4.1-Fast*** демонстрирует идеальную точность во всех сценариях, корректно определяя типы ландшафтов от сложных городских развязок до природных зон.

***Phi-4-Multimodal*** показывает отличные результаты, лишь незначительно уступая лидеру. Модель точно распознаёт урбанизированные зоны, прибрежные территории и сельскохозяйственные угодья. Единственная небольшая неточность — определение лесной развязки как "городской местности" (Изображение 4).

***Nemotron-Nano-12B-VL*** часто допускает ошибки: неправильно классифицирует хвойный лес (Изображение 1, 14), даёт слишком общие определения ("контрастная местность" вместо конкретного описания) или использует нерелевантные термины вроде "дорожная инфраструктура" вместо конкретного типа местности.

### Полнота обнаруженных объектов

***Grok-4.1-Fast*** показывает лучший результат (4.80), последовательно обнаруживая все значимые объекты: здания, дороги, транспорт, водоёмы, растительность и инфраструктурные элементы.

***Phi-4-Multimodal*** демонстрирует сопоставимые показатели (4.60), успешно идентифицируя ключевые объекты на изображениях.

***Nemotron-Nano-12B-VL*** часто пропускает важные объекты или указывает несуществующие (например, "испытательный полигон" на городском перекрёстке, "еноти" в жилом районе, "цели(ны) животных" и "кота" на сельскохозяйственных полях, странные объекты вроде "бра нас протектор").

### Детальность описания

***Grok-4.1-Fast*** создаёт максимально информативные описания с упоминанием контекста: сезонность ("осенняя листва"), характер планировки ("радиальная планировка"), функциональное назначение ("многоуровневая автодорожная развязка").

***Phi-4-Multimodal*** предоставляет хорошие описания (средний балл 4.33), но менее детальные. Описания точны и информативны, однако иногда более лаконичны по сравнению с Grok.

***Nemotron-Nano-12B-VL*** создаёт самые слабые описания: часто слишком общие, содержат непонятные термины, иероглифы или нечитаемые фразы (например, описание Изображения 14 совершенно бессмысленно, а в описании Изображения 15 упоминаются несуществующие "лучи снизу").

### Производительность

***Phi-4-Multimodal*** — безусловный лидер по скорости (медиана 3.46 сек), опережая конкурентов в 2-4 раза. Особенно впечатляют результаты на сложных изображениях: 2.01 сек для моста, 2.63 сек для прибрежной зоны, 2.05 сек для тропического пляжа.

***Nemotron-Nano-12B-VL*** показывает среднюю скорость (медиана 6.82 сек), что в 2 раза медленнее Phi-4.

***Grok-4.1-Fast*** работает медленнее всего (медиана 13.65 сек), но обеспечивает максимальное качество анализа.


### Итоговая рекомендация

**Для задач, где критично качество**: ***Grok-4.1-Fast*** — идеальный выбор с максимально точными и детальными результатами.

**Для задач, где важен баланс**: ***Phi-4-Multimodal*** — отличное сочетание высокого качества (4.62 из 5.00) и скорости, превосходя Grok по производительности более чем в 4 раза.

**Nemotron-Nano-12B-VL** — наименее подходящая модель из трёх, уступающая конкурентам как по качеству, так и по скорости.